In [3]:
import os
import json
import pandas as pd
import zipfile
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# print(tf.config.list_physical_devices("GPU"))
data = pd.read_csv("../data/IMDB_reviews.csv")

data.head()


2025-06-24 13:40:24.708068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-24 13:40:24.728052: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-24 13:40:24.734300: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-24 13:40:24.749834: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-24 13:40:25.103483: W tensorflow/compiler/tf2

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
# data.shape


data["sentiment"].value_counts()


sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [5]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

data["sentiment"].value_counts()
# data.head()


/tmp/ipykernel_906/3893172435.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


sentiment
1    25000
0    25000
Name: count, dtype: int64

In [6]:
train_data, test_data = train_test_split(
    data, random_state=42, shuffle=True, test_size=0.1
)


tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])

X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

y_train = train_data["sentiment"]
y_test = test_data["sentiment"]


In [7]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


/home/mrmauler/DRIVE/projects/dl/sentiment_analysis/sentiment_analysis_env/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1750772441.986183     906 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1750772435.154941     906 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1750772435.155038     906 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1750772435.157779     906 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built wi

In [8]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.fit(X_train, y_train, epochs=3, batch_size=64, validation_split=0.2)

# loss = model.evaluate(X_test, y_test)


Epoch 1/3
563/563 ━━━━━━━━━━━━━━━━━━━━ 904s 2s/step - accuracy: 0.7334 - loss: 0.5180 - val_accuracy: 0.8110 - val_loss: 0.4318
Epoch 2/3
563/563 ━━━━━━━━━━━━━━━━━━━━ 905s 2s/step - accuracy: 0.8398 - loss: 0.3780 - val_accuracy: 0.8211 - val_loss: 0.4364
Epoch 3/3
563/563 ━━━━━━━━━━━━━━━━━━━━ 846s 2s/step - accuracy: 0.8586 - loss: 0.3440 - val_accuracy: 0.8587 - val_loss: 0.3434


In [10]:
loss, accuracy = model.evaluate(X_test, y_test)

print(f"Loss :{loss}")
print(f"Accuracy :{accuracy}")


157/157 ━━━━━━━━━━━━━━━━━━━━ 36s 232ms/step - accuracy: 0.8581 - loss: 0.3408
Loss :0.3420836329460144
Accuracy :0.8560000061988831


In [11]:
def predict_sentiment(review):
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=200)
    # padded_sequence = pad_sequences(tokenizer.texts_to_sequences(review), maxlen=200)
    prediction = model.predict(padded_sequence)

    sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
    return sentiment


In [12]:
rev = "that movie was great"

print(predict_sentiment(rev))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 704ms/step
positive


In [19]:
import joblib
from tensorflow.keras.models import save_model


# model.save("model.keras")
joblib.dump(tokenizer, "tokenizer.pkl", protocol=2)

save_model(model, "model.keras", save_format="keras_v3")
